In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 31.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [ ]:
from ortools.linear_solver import pywraplp

def prisoners_dilemma(num_rounds):
    # Define the payoffs
    payoff = [[(2, 2), (0, 5)],
              [(5, 0), (3, 3)]]

    # Create the solver
    solver = pywraplp.Solver.CreateSolver('GLOP')

    # Create variables
    player1_strategy = [solver.BoolVar('Player1_strategy{}'.format(i+1)) for i in range(num_rounds)]
    player2_strategy = [solver.BoolVar('Player2_strategy{}'.format(i+1)) for i in range(num_rounds)]

    # Set objective function
    objective = solver.Objective()
    for i in range(num_rounds):
        objective.SetCoefficient(player1_strategy[i], payoff[0][0][1] if i == 0 else payoff[0][1][1])
        objective.SetCoefficient(player2_strategy[i], payoff[1][0][1] if i == 0 else payoff[1][1][1])
    objective.SetMaximization()

    # Add constraints
    for i in range(num_rounds):
        constraint = solver.Constraint(1, 1)
        constraint.SetCoefficient(player1_strategy[i], 1)
        constraint.SetCoefficient(player2_strategy[i], 1)

        if i > 0:
            constraint = solver.Constraint(0, 1)
            constraint.SetCoefficient(player1_strategy[i], 1)
            constraint.SetCoefficient(player1_strategy[i - 1], 1)

            constraint = solver.Constraint(0, 1)
            constraint.SetCoefficient(player2_strategy[i], 1)
            constraint.SetCoefficient(player2_strategy[i - 1], 1)

    # Solve the problem
    solver.Solve()

    # Print the results
    print('Player 1 strategy:')
    for i in range(num_rounds):
        strategy = 'Cooperate' if player1_strategy[i].solution_value() else 'Betrays'
        print('Round {}: {}'.format(i+1, strategy))

    print('Player 2 strategy:')
    for i in range(num_rounds):
        strategy = 'Cooperate' if player2_strategy[i].solution_value() else 'Betrays'
        print('Round {}: {}'.format(i+1, strategy))

    print('Player 1 payoff:', objective.Value())
    print('Player 2 payoff:', num_rounds - objective.Value())

# Run the simulation for 5 rounds
prisoners_dilemma(5)

Player 1 strategy:
Round 1: Cooperate
Round 2: Betrays
Round 3: Cooperate
Round 4: Betrays
Round 5: Cooperate
Player 2 strategy:
Round 1: Betrays
Round 2: Cooperate
Round 3: Betrays
Round 4: Cooperate
Round 5: Betrays
Player 1 payoff: 18.0
Player 2 payoff: -13.0
